In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
cities_50km_rad = dataiku.Dataset("cities_50km_rad")
cities_50km_rad_df = cities_50km_rad.get_dataframe()


In [0]:
cities_50km_rad_df

In [0]:

import pandas as pd
import concurrent.futures
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Initialize geocoder
geolocator = Nominatim(user_agent="geocoding_app")

# Multithreaded geocoding function with retry on timeout
def get_lat_long(location, retries=3):
    try:
        location_info = geolocator.geocode(location, timeout=10)
        if location_info:
            return location, location_info.latitude, location_info.longitude
        else:
            return location, None, None
    except GeocoderTimedOut:
        if retries > 0:
            time.sleep(1)  # Wait a bit before retrying
            return get_lat_long(location, retries - 1)
        else:
            print(f"Timeout geocoding {location}")
            return location, None, None
    except Exception as e:
        print(f"Error geocoding {location}: {e}")
        return location, None, None

def add_lat_long_columns_multithreaded(df, location_column):
    locations = df[location_column].unique()
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(get_lat_long, loc): loc for loc in locations}
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    lat_long_map = {loc: (lat, lon) for loc, lat, lon in results}

    df[['latitude', 'longitude']] = df[location_column].apply(
        lambda x: pd.Series(lat_long_map.get(x, (None, None)))
    )
    return df




In [0]:
# Apply geocoding with multithreading
cities_50km_rad_df = add_lat_long_columns_multithreaded(cities_50km_rad_df, 'Nearby_City_within_50km')



In [0]:
cities_50km_rad_df

In [0]:
def add_lat_long_columns_multithreaded_City(df, location_column):
    locations = df[location_column].unique()
    results = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(get_lat_long, loc): loc for loc in locations}
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())

    lat_long_map = {loc: (lat, lon) for loc, lat, lon in results}

    df[['latitude_City', 'longitude_City']] = df[location_column].apply(
        lambda x: pd.Series(lat_long_map.get(x, (None, None)))
    )
    return df

In [0]:
cities_50km_rad_df = add_lat_long_columns_multithreaded_City(cities_50km_rad_df, 'City')


In [0]:
cities_50km_rad_df.columns

In [0]:
order = ['City', 'latitude_City', 'longitude_City', 'Nearby_City_within_50km', 'latitude', 'longitude']

In [0]:
df = cities_50km_rad_df[order]

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

city_lat_long_df = df # For this sample code, simply copy input to output


# Write recipe outputs
city_lat_long = dataiku.Dataset("city_lat_long")
city_lat_long.write_with_schema(city_lat_long_df)